In [4]:
# numpy,pandas
import numpy as np
import pandas as pd
# vis
import seaborn as sns
import math
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
# kr
font_path = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)
# stats
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import anderson
from scipy.stats import bartlett
from scipy.stats import levene
from scipy.stats import pearsonr
import statsmodels.api as sm
from statsmodels.formula.api import ols
# 원하는 csv파일을 불러옵니다.
df = pd.read_csv('데이터 찐최종본(완).csv')
pd.set_option('display.max_rows', None)

# 새로운 열 이름을 생성하고 열 이름을 변경합니다.
new_columns = ['강좌명','수강생수','가격','평점','수강평수','난이도','강의시간','강의게시일']
df.columns = new_columns


# 각각의 속성을 원하는 시각화를 위해 형태를 바꿔줍니다. -> 시각화를 위한 데이터프레임을 생성합니다.
df['강좌명'] = df['강좌명'].str.replace('\n대시보드', '')
df['강의게시일'] = pd.to_datetime(df['강의게시일'].str.extract(r'(\d+)년 (\d+)월 (\d+)일').agg('-'.join, axis=1))
df['수강평수'] = df['수강평수'].str.extract(r'(\d+)개의').fillna(0).astype(int)
df['수강생수'] = df['수강생수'].str.replace('명', '')
df['수강생수'] = df['수강생수'].str.replace(',', '').astype(int)
df['가격'] = df['가격'].replace('무료', '₩0')
df['가격'] = df['가격'].str.extract(r'₩([^₩]+)$')
df['가격'] = df['가격'].str.replace(',', '').astype(int)
df['시간'] = df['강의시간'].str.extract(r'(\d+)시간', expand=False).fillna(0)
df['분'] = df['강의시간'].str.extract(r'(\d+)분', expand=False).fillna(0)
df['강의시간_분'] = df['시간'].astype(int)*60 + df['분'].astype(int)
df = df.drop(['시간', '분', '강의시간'], axis=1)
df['평점'] = df['평점'].str.extract(r'([\d.]+)').fillna(0).astype(float)
df['무료/유료'] = df['가격'].apply(lambda x: '무료' if x == 0 else '유료')
df['평가지수'] = df['수강평수'] * df['평점']
# 중복되는 행을 제거해줍니다.
df = df.drop_duplicates('강좌명', keep='first')



# 회귀 분석을 위한 데이터프레임을 생성합니다.
df_regression = df.copy()
df_regression = pd.concat([df_regression, pd.get_dummies(df_regression['난이도'], prefix='난이도')], axis=1)
df_regression['유무료'] = df['무료/유료'].apply(lambda x: 1 if x == '유료' else 0) #유료(1)/무료(0)
df_regression= df_regression.drop(['난이도', '무료/유료'], axis=1)

In [5]:
df1=df_regression.copy()
df1 = df1[df1['평점'] != 0]
df1 = df1[df1['강의시간_분'] != 0]
print(len(df1.loc[df1['평점'] == 0]))
print(len(df1.loc[df1['강의시간_분'] == 0]))
df1.info()

0
0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1703 entries, 0 to 2786
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   강좌명      1703 non-null   object        
 1   수강생수     1703 non-null   int32         
 2   가격       1703 non-null   int32         
 3   평점       1703 non-null   float64       
 4   수강평수     1703 non-null   int32         
 5   강의게시일    1703 non-null   datetime64[ns]
 6   강의시간_분   1703 non-null   int32         
 7   평가지수     1703 non-null   float64       
 8   난이도_입문자  1703 non-null   uint8         
 9   난이도_중급자  1703 non-null   uint8         
 10  난이도_초급자  1703 non-null   uint8         
 11  유무료      1703 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int32(4), int64(1), object(1), uint8(3)
memory usage: 111.4+ KB


In [6]:
# outliers remeove
def remove_outliers_iqr(df1, column):
    Q1 = df1[column].quantile(0.25)
    Q3 = df1[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df1[(df1[column] >= lower_bound) & (df1[column] <= upper_bound)].reset_index(drop=True)

columns_to_remove_outliers = ['수강생수', '가격', '강의시간_분']
for column in columns_to_remove_outliers:
    df1 = remove_outliers_iqr(df1, column)